<a href="https://colab.research.google.com/github/JRSmiffy/TensorFlow-Practise/blob/master/Regression_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Import the training data

In [0]:
import pandas as pd

In [0]:
housing = pd.read_csv('https://raw.githubusercontent.com/JRSmiffy/TensorFlow-Practise/master/cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


### Step 2: Perform the Train-Test Split

In [0]:
y_val = housing['medianHouseValue'] # This is what we are trying to predict

In [0]:
x_data = housing.drop('medianHouseValue', axis = 1 ) # This is the remainder of the features - used for prediction

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_val, test_size = 0.3, random_state = 101)

In [8]:
print(x_train.size) # 70% of x_data
print(x_test.size) # 30% of ""
print(y_train.size) # 70% of y_val
print(y_test.size) # 30% of ""
# y_val.size is 20639 (1 col), x_data.size is 6 * y_val (6 col) & housing = x _data + y_val

86688
37152
14448
6192


### Step 3: Scale the Feature Data

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [0]:
x_train = pd.DataFrame(data=scaler.transform(x_train), columns = x_train.columns, index = x_train.index)

In [0]:
x_test = pd.DataFrame(data=scaler.transform(x_test), columns = x_test.columns, index = x_test.index)

In [14]:
print(x_train) # How have we scaled the data? - zero mean & unit variance, why though?

       housingMedianAge  totalRooms  ...  households  medianIncome
6761           0.352941    0.069688  ...    0.115442      0.142508
3010           0.607843    0.011242  ...    0.014142      0.045027
7812           0.666667    0.025230  ...    0.030258      0.212866
8480           0.666667    0.032530  ...    0.030094      0.298651
1051           0.294118    0.031919  ...    0.034863      0.272631
16312          0.607843    0.113739  ...    0.145206      0.192342
2042           0.431373    0.053283  ...    0.066436      0.207783
1755           0.882353    0.048298  ...    0.057721      0.346181
16022          1.000000    0.061015  ...    0.053774      0.436704
20441          0.450980    0.103515  ...    0.092748      0.382671
13243          0.156863    0.098555  ...    0.080414      0.550613
6450           0.843137    0.050994  ...    0.050321      0.420615
10229          0.686275    0.023628  ...    0.049663      0.148715
6463           0.823529    0.046569  ...    0.054761      0.21

### Step 4: Create the Feature Columns - Transform the data into formats the estimators can use

In [15]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [0]:
import tensorflow as tf

In [0]:
age = tf.feature_column.numeric_column('housingMedianAge')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [0]:
feat_cols = [age, bedrooms, pop, households, income]

### Step 5: Create Input Function for Estimator Object

In [0]:
input_func = tf.estimator.inputs.pandas_input_fn(x = x_train, y = y_train, batch_size = 10, num_epochs = 1000, shuffle = True)

### Step 6: Create the Estimator Model (DNN Regressor)

In [20]:
model = tf.estimator.DNNRegressor(hidden_units=[12,12,12], feature_columns = feat_cols)

W0708 15:29:16.359970 140113151928192 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpn3p_77e8


### Step 7: Train the Model

In [21]:
model.train(input_fn = input_func, steps = 1000)

W0708 15:29:16.402688 140113151928192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0708 15:29:16.426156 140113151928192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0708 15:29:16.428963 140113151928192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_functions.py:500: add_

### Step 8: Create the prediction function

In [0]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x = x_test, batch_size = 10, num_epochs = 1, shuffle = False)

In [0]:
pred_gen = model.predict(predict_input_func)

In [24]:
predictions = list(pred_gen)

W0708 15:29:21.695214 140113151928192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [25]:
predictions

[{'predictions': array([226721.52], dtype=float32)},
 {'predictions': array([268442.8], dtype=float32)},
 {'predictions': array([225401.06], dtype=float32)},
 {'predictions': array([178898.11], dtype=float32)},
 {'predictions': array([238273.14], dtype=float32)},
 {'predictions': array([198042.64], dtype=float32)},
 {'predictions': array([232403.42], dtype=float32)},
 {'predictions': array([207374.52], dtype=float32)},
 {'predictions': array([205343.73], dtype=float32)},
 {'predictions': array([166435.94], dtype=float32)},
 {'predictions': array([206032.05], dtype=float32)},
 {'predictions': array([222630.42], dtype=float32)},
 {'predictions': array([193869.3], dtype=float32)},
 {'predictions': array([177864.4], dtype=float32)},
 {'predictions': array([259187.45], dtype=float32)},
 {'predictions': array([171751.77], dtype=float32)},
 {'predictions': array([209481.83], dtype=float32)},
 {'predictions': array([179868.14], dtype=float32)},
 {'predictions': array([172120.67], dtype=float32

### Step 9: Calculate the RMSE

In [0]:
final_preds = []

for pred in predictions:
    final_preds.append(pred['predictions'])

In [0]:
from sklearn.metrics import mean_squared_error

In [28]:
mean_squared_error(y_test, final_preds)**0.5 # Is this the error in our model? i.e the value we want to minimize...

106006.88088581307